In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

In [3]:
import pandas as pd
# Load the data
data = pd.read_csv('umsatz.csv')
data

,Kundengruppe,Artikelgruppe,Region,Umsatz
0,Fachhandel,Tourenräder,West,niedrig
1,Discounter,Cityräder,West,hoch
2,Fachhandel,Mountainbikes,Nord,mittel
3,Fachhandel,Tourenräder,Nord,niedrig
4,Fachhandel,Tourenräder,Süd,hoch
5,Kaufhaus,Mountainbikes,Nord,mittel
6,Discounter,Mountainbikes,West,mittel
7,Discounter,Cityräder,West,hoch
